# Italian to English Language Translation (NMT) using GRU & Attention

**Import All the neccessary Libraries which we will be requiring to run this notebook and the project**

This is seq2seq model by using GRU and Attention. let us import all the necessary libraries that is needed to run this notebook

the tensorflow GPU version must be updated before running this notebook and eager_execution of the tensorflow should be enabled

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
from numpy import array, argmax, random, take
import time
import pandas as pd
import string
print(tf.__version__)

1.13.1


Creating a function which will read the file, encode it and save it and then the funtion to_lines wil  split the data into Italian & English part seperately by '\n' and build it in the form of sentences

In [0]:
def read_text(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

# split a text into sentences
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t') for i in sents]
    return sents

In [3]:
   from google.colab import files

   uploaded = files.upload()

Saving ita.txt to ita.txt


In [6]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "ita.txt" with length 19184787 bytes


By using the function we wrote the text file is imported to the jupyter notebook. 

Once the notebook is opened and read using the function Read_TextFile we will pass it to the to_lines to split and to build the sentences of the same

In [0]:
data = read_text(uploaded['ita.txt'])
ItalianNEng = to_lines(data)
ItalianNEng

The ItalianEng has the data which we just imported and then ran through a funtion is now converted to the array by using the Python's inbuilt function "array"

In [0]:
data = read_text("ita.txt")
ItalianNEng = to_lines(data)
ItalianNEng = array(ItalianNEng)

## PreProcessing the data/ Cleaning the data

**remove all the punctuation and then change the case of every word to lower case**


we will remove the punctuation in the below code by going through each line of the data and storing it in the same variable ItalianEng

In [0]:
# Remove punctuation
ItalianNEng[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in ItalianNEng[:,0]]
ItalianNEng[:,1] = [s.translate(str.maketrans('', '', string.punctuation)) for s in ItalianNEng[:,1]]

Here all the punctuation is removed/cleaned and the data looks like below

In [10]:
ItalianNEng

array([['Hi', 'Ciao'],
       ['Run', 'Corri'],
       ['Run', 'Corra'],
       ...,
       ['If you want to sound like a native speaker you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo',
        'Se vuoi sembrare un madrelingua devi essere disposto a esercitarti a ripetere la stessa frase più e più volte nello stesso modo in cui i suonatori di banjo praticano ripetutamente la stessa frase fino a che non riescono a suonarla correttamente e al tempo desiderato'],
       ['If someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker In other words you dont really sound like a native speaker',
        'Se qualcuno che non conosce il tuo background dice che sembri un madrelingua significa che probabilment

Now we will convert the data into lower case by using python's inbuilt function lower() and save the data to its original variable ItalianEng

In [0]:
# convert to lowercase
for i in range(len(ItalianNEng)):
    ItalianNEng[i,0] = ItalianNEng[i,0].lower()
    
    ItalianNEng[i,1] = ItalianNEng[i,1].lower()

In [12]:
ItalianNEng

array([['hi', 'ciao'],
       ['run', 'corri'],
       ['run', 'corra'],
       ...,
       ['if you want to sound like a native speaker you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo',
        'se vuoi sembrare un madrelingua devi essere disposto a esercitarti a ripetere la stessa frase più e più volte nello stesso modo in cui i suonatori di banjo praticano ripetutamente la stessa frase fino a che non riescono a suonarla correttamente e al tempo desiderato'],
       ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker',
        'se qualcuno che non conosce il tuo background dice che sembri un madrelingua significa che probabilment

In [13]:
ItalianNEng[0][0]

'hi'

we convert the data into pandas dataframe and save it in the variable ita_eng_df

In [14]:
ita_eng_df = pd.DataFrame(ItalianNEng.tolist(), columns = ['English', 'Italian'])
ita_eng_df.head()

,English,Italian
0,hi,ciao
1,run,corri
2,run,corra
3,run,correte
4,who,chi


let us split the data into English and Italian part seperately and save it in their respective variable

In [15]:
english_sentences = ita_eng_df.iloc[:,0:1]
italian_sentences = ita_eng_df.iloc[:,1:2]
print(english_sentences.columns)
print(italian_sentences.columns)

Index(['English'], dtype='object')
Index(['Italian'], dtype='object')


As the dataset is very large we will sample the model with only 2000 rows, later we can play around the data size and see how the model performs

In [0]:
n = 4000
italian_sample = italian_sentences.iloc[:n,:]
english_sample = english_sentences.iloc[:n,:]

we will now convert any row which is not of string type, if there is any row with different datatype we will convert it into string so that everything is of same datatype


we will do this for both English and Italian data

In [0]:
english_sample.columns
english_sample['English']
for i in range(len(english_sample['English'].index)):
    if type(english_sample['English'][i]) != str:
        english_sample['English'][i] = str(english_sample['English'][i])
        
italian_sample['Italian']
for i in range(len(italian_sample['Italian'].index)):
    if type(italian_sample['Italian'][i]) != str:
        italian_sample['Italian'][i] = str(italian_sample['Italian'][i])

create a fucntion to add start and end token of each row as a identifier to the model and convert the data into lower case

In [0]:
def preprocess_sentence(w):
    w = w.lower().strip()
    w = '<start> ' + w + ' <end>'
    return w

def cleaning_sentence(data, column):
    sentence = [preprocess_sentence(i) for i in data[column]]
    return sentence


We will continue to create a class which has function which will create word to index and then index to word

In [0]:
class LanguageIndex():    
    def __init__(self, lang):        
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
          self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
          self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
          self.idx2word[index] = word   
    


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

**Cleaning the data**

we will make use of the funciton such as cleaning_sentence, LanguageIndex, and word2idx which we have written to clean, process, Index and padd the data


we use Keras's inbuil function which performs sequence padding to the each sentence to the maximum data length

In [0]:
#Cleaning Sentences
italian_sent = cleaning_sentence(italian_sample, 'Italian')
english_sent = cleaning_sentence(english_sample, 'English')

# index language using the class defined above
inp_lang = LanguageIndex(it for it in italian_sent)
targ_lang = LanguageIndex(en for en in english_sent)

#Italian Sentences which will be indexed
input_tensor = [[inp_lang.word2idx[s] for s in it.split(' ')] for it in italian_sent]
#English sentences which will be indexed
target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en in english_sent]

max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

# Padding the input and output tensor to the maximum length
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                             maxlen=max_length_inp,
                                                             padding='post')

target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                              maxlen=max_length_tar, 
                                                              padding='post')    

**split the data into training data and test data by using Sklearn.Model_selection's train_test_split function**


Split the whole data which we saved in the variable ItalianNEng as 80% to train and 20% to the test.

In [0]:
ax_train, ay_train, ax_test, ay_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

after we split the training data and the testing data we will check the length and the data in it, if we see the below data its vectorized which we did using the word2idx and Keras's inbuilt function sequence_padding and is in the array format

In [23]:
ax_train

array([[   6,  844, 1554, ...,    0,    0,    0],
       [   6, 1885, 1142, ...,    0,    0,    0],
       [   6,  178,    5, ...,    0,    0,    0],
       ...,
       [   6,  498,    5, ...,    0,    0,    0],
       [   6,  844,  988, ...,    0,    0,    0],
       [   6, 1492, 1130, ...,    0,    0,    0]], dtype=int32)

In [24]:
# Show length
len(ax_train), len(ay_train), len(ax_test), len(ay_test)

(3200, 800, 3200, 800)

Create the parameters where we can modify and pass later to tune the model and we will create the tensorflow dataset using the tf's function tf.data.dataset

In [25]:
BUFFER_SIZE = len(ax_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)


print(BUFFER_SIZE)
print(BATCH_SIZE)
print(N_BATCH)
print(vocab_inp_size)
print(vocab_tar_size)

3200
64
50
1886
693


In [26]:
dataset = tf.data.Dataset.from_tensor_slices((ax_train, ax_test)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 9), (64, 6)), types: (tf.int32, tf.int32)>

We will build the encoder and decoder model by implementing "attention equation"

attention overcomes the limitation in the encode-decoder architecture by allowing the network to learn where to pay attention to the input for each item in the output sequence.

this approach has been used across different types sequence prediction problems include text translation, speech recognition, and more.

In [0]:
def gru(units):
    return tf.keras.layers.GRU(units, 
                           return_sequences=True, 
                           return_state=True, 
                           recurrent_activation='sigmoid', 
                           recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights

In [0]:
def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

**Optimizer and Loss Function**

We are using "Adam" as the optimizer for the model and sparse_softmax_cross_entropy as the loss function.

we can play around with these to see which is better suitable for our model, but to train the model we will be going with the above mentoned optimizer and the loss function

In [0]:
optimizer = tf.train.AdamOptimizer()
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

## Training the model

we will train the model by passing the training data which we split and passed it through the funtion. we will be saving out model on each iteration of the epochs 

We will be running this model with **30 Epochs**

we will play around with the number of epochs to see what best works for the model

In [33]:
EPOCHS = 40

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))   
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1 Batch 0 Loss 3.6454
Epoch 1 Loss 2.5360
Time taken for 1 epoch 9.944748163223267 sec

Epoch 2 Batch 0 Loss 1.9223
Epoch 2 Loss 1.8720
Time taken for 1 epoch 8.805859565734863 sec

Epoch 3 Batch 0 Loss 1.7307
Epoch 3 Loss 1.7082
Time taken for 1 epoch 9.48662543296814 sec

Epoch 4 Batch 0 Loss 1.5072
Epoch 4 Loss 1.5026
Time taken for 1 epoch 8.954044580459595 sec

Epoch 5 Batch 0 Loss 1.4262
Epoch 5 Loss 1.2997
Time taken for 1 epoch 8.833491802215576 sec

Epoch 6 Batch 0 Loss 1.1589
Epoch 6 Loss 1.1484
Time taken for 1 epoch 8.69847846031189 sec

Epoch 7 Batch 0 Loss 1.0229
Epoch 7 Loss 1.0363
Time taken for 1 epoch 8.853046417236328 sec

Epoch 8 Batch 0 Loss 0.9543
Epoch 8 Loss 0.9238
Time taken for 1 epoch 8.750823974609375 sec

Epoch 9 Batch 0 Loss 0.7996
Epoch 9 Loss 0.8197
Time taken for 1 epoch 8.902754068374634 sec

Epoch 10 Batch 0 Loss 0.7619
Epoch 10 Loss 0.7445
Time taken for 1 epoch 8.692206859

## Prediction
Once the model is trained we can translate the Italian sentences by passing it to the below function

In [0]:
def Prediction_eval(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = Prediction_eval(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    

In [36]:
english_sample.iloc[:1,:].values[0][0]

'hi'

In [37]:
translate(italian_sample.iloc[:1,:].values[0][0], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_tar)


Input: <start> ciao <end>
Predicted translation: we walked <end> 


In [42]:
test = 'venire'
translate(sentence=test, encoder=encoder, decoder=decoder, inp_lang= inp_lang, targ_lang= targ_lang, max_length_inp= max_length_inp, max_length_targ= max_length_tar )

Input: <start> venire <end>
Predicted translation: come over <end> 


## References


**o** https://machinelearningmastery.com/attention-long-short-term-memory-recurrent-neural-networks/

**o** https://medium.com/@ageitgey/machine-learning-is-fun-part-5-language-translation-with-deep-learning-and-the-magic-of-sequences-2ace0acca0aa

**o** https://www.analyticsvidhya.com/blog/2018/03/microsofts-claims-language-translation-ai-reached-human-levels-accuracy/

**o** https://blog.statsbot.co/machine-learning-translation-96f0ed8f19e4

**o** https://www.youtube.com/watch?v=nRBnh4qbPHI&vl=en

**o** http://www.manythings.org/anki/

**o** https://machinelearningmastery.com/introduction-neural-machine-translation/

**o** https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/

**o** https://www.youtube.com/watch?v=vI2Y3I-JI2Q

**o** https://towardsdatascience.com/neural-machine-translator-with-less-than-50-lines-of-code-guide-1fe4fdfe6292

**o** https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/text/

**o** https://machinelearningmastery.com/tune-lstm-hyperparameters-keras-time-series-forecasting/



## License

**MIT License**

Copyright 2018 Chetan M Jadhav

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

**The text in the document by Chetan M Jadhav is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/ **